In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

## The model

In [2]:
model = pyo.ConcreteModel()

## The sets

In [3]:
projects: list[str] = ["P1", "P2", "P3"]
model.projects = pyo.Set(initialize=projects)

In [4]:
model.projects.display()

projects : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'P1', 'P2', 'P3'}


## Parameters

In [5]:

model.ProjectManagement = pyo.Param(model.projects, initialize={"P1": 18, "P2": 23, "P3": 15})
model.DataModeling = pyo.Param(model.projects, initialize={"P1": 4, "P2": 0, "P3": 20})
model.DataScience = pyo.Param(model.projects, initialize={"P1": 12, "P2": 30, "P3": 100})
model.Priorities = pyo.Param(model.projects, initialize={"P1": 3, "P2": 1, "P3": 2})

## Decision variables

In [6]:
model.x = pyo.Var(model.projects, within=pyo.NonNegativeIntegers)

## Objective functions

In [7]:
def objective_callback(model):
    return sum(model.Priorities[p] * model.x[p] for p in model.projects)

In [8]:
model.ObjectiveFunction = pyo.Objective(rule=objective_callback, sense=pyo.maximize)

## Constraints

In [9]:
def project_management_constraint_callback(model: pyo.Model, i):
    """
    Parameters:
    model: The model
    i: It is important to specify this parameter in case we have multiple sets. Otherwise we would have an ambiguity.
    """
    return sum(model.ProjectManagement[p] * model.x[p] for p in model.projects) <= 60


model.ConstraintProjectManagement = pyo.Constraint(model.projects, rule=project_management_constraint_callback)


def data_modeling_constraint_callback(model: pyo.Model, i):
    return sum(model.DataModeling[p] * model.x[p] for p in model.projects) <= 48


model.ConstraintDataModeling = pyo.Constraint(model.projects, rule=data_modeling_constraint_callback)


def data_science_constraint_callback(model: pyo.Model, i):
    return sum(model.DataScience[p] * model.x[p] for p in model.projects) <= 60


model.ConstraintDataScience = pyo.Constraint(model.projects, rule=data_science_constraint_callback)


def project_outcome_constraint_callback(model: pyo.Model, project):
    return model.x[project] <= 1


model.ConstraintOutcome = pyo.Constraint(model.projects, rule=project_outcome_constraint_callback)

## Solve the model

In [11]:
solver = SolverFactory("glpk")
results = solver.solve(model)

print(results)

print(f"Objective function =  {model.ObjectiveFunction()}")

for p in model.projects:
    print(f"Is {p} prioritized? {model.x[p]()}={'Yes' if model.x[p]() == 1  else 'No'}")


Problem: 
- Name: unknown
  Lower bound: 4.0
  Upper bound: 4.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 4
  Number of nonzeros: 28
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06656026840209961
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective function =  4.0
Is P1 prioritized? 1.0=Yes
Is P2 prioritized? 1.0=Yes
Is P3 prioritized? 0.0=No
